Test 3 ResNet50

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip" -d "/content/data"

In [ ]:
!unzip "/content/drive/MyDrive/Dataset/data.zip"

In [4]:
import warnings

# Ignore warnings
warnings.filterwarnings("ignore")